# MOVIE RECOMMENDER SYSTEM


Importing the dependencies

In [ ]:
import numpy as np import pandas as pd

In [ ]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

since credits dataset contain only 4 columns, we can merge it into single dataframe with movies

In [86]:
movies=movies.merge(credits,on='title')

WE will only choose those columns which are useful to us and will discard the others.

In [87]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

we will check or dataset for any null values

In [88]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

since overview column has only 3 null values which a very small number, so we can simply drop them 

In [89]:
movies.dropna(inplace=True)

In [90]:
movies['overview'].isnull().sum()

0

In [91]:
import ast

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L
movies['genres']= movies['genres'].apply(convert)

In [92]:

movies['keywords'] = movies['keywords'].apply(convert)
#movies['keywords'] = movies['keywords'].apply(convert)


In [93]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L
movies['cast'] = movies['cast'].apply(convert3)


In [94]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [95]:
import ast

def convert4(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job']=='Director':
            L.append(i['name'])
            
    return L
movies['crew']=movies['crew'].apply(convert4)


In [96]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [97]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [98]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [99]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [100]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [101]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [102]:

new['tags'] = new['tags'].apply(lambda x: " ".join(x))

In [103]:
new['tags'] = new['tags'].apply(lambda x: x.lower())

In [104]:
new.head(1)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."


In [105]:
from sklearn.feature_extraction.text import CountVectorizer

In [106]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [107]:
vector= cv.fit_transform(new['tags']).toarray()

In [108]:
cv.get_feature_names()

['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '18th',
 '19',
 '1930s',
 '1940s',
 '1944',
 '1950',
 '1950s',
 '1960s',
 '1970s',
 '1971',
 '1974',
 '1976',
 '1980',
 '1980s',
 '1985',
 '1990s',
 '1999',
 '19th',
 '19thcentury',
 '20',
 '200',
 '2003',
 '2009',
 '20th',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '60',
 '60s',
 '70',
 'aaron',
 'aaroneckhart',
 'abandoned',
 'abducted',
 'abigailbreslin',
 'abilities',
 'ability',
 'able',
 'aboard',
 'abuse',
 'abusive',
 'academic',
 'academy',
 'accept',
 'accepted',
 'accepts',
 'access',
 'accident',
 'accidental',
 'accidentally',
 'accompanied',
 'accomplish',
 'account',
 'accountant',
 'accused',
 'ace',
 'achieve',
 'act',
 'acting',
 'action',
 'actionhero',
 'actions',
 'activist',
 'activities',
 'activity',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'adam',
 'adams',
 'adamsandler',
 'adamshankman',
 'adaptation',
 'adapted',
 'addict',
 'addi

In [109]:
from nltk.stem.porter import PorterStemmer

In [110]:
stemmer = PorterStemmer()

In [111]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(stemmer.stem(i))
    return " ".join(y)
new['tags']=new['tags'].apply(stem)

In [112]:
#Euclidean distance does not work well in high dimensional data so we use cosine distance instead 

from sklearn.metrics.pairwise import cosine_similarity

In [113]:

similarity = cosine_similarity(vector)

In [114]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)
recommend('Avatar')
        

Titan A.E.
Small Soldiers
Ender's Game
Aliens vs Predator: Requiem
Independence Day


In [115]:
import pickle

In [116]:
pickle.dump(new,open('movies.pkl','wb'))

In [117]:
new['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [118]:
new_dict=new.to_dict()

In [119]:
pickle.dump(new_dict,open('movies_dict.pkl','wb'))

In [120]:
pickle.dump(similarity,open('similarity.pkl','wb'))